<a href="https://colab.research.google.com/github/dhinton22/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,336 kB]
Hit:13 https://developer.dow

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-09 01:31:46--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.59MB/s    in 0.6s    

2022-08-09 01:31:48 (1.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [37]:
# Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)

In [38]:
# Step 2: Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
percent_votes_df = total_votes_df.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).alias('percent_votes').filter(col("percent_votes") >= 0.5)
percent_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|0.6923076923076923|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|0.6153846153846154|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|              0.64|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|0.9344262295081968|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|               1.0|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|0.9072164948453608|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|0.6744186046511628|
|R39OOYN0DOLINK|          3|  

In [39]:
# Step 3: Retrieves all the rows where a review was written as part of the Vine program (paid)
paid_df = percent_votes_df.filter(col("vine") == "Y").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R28EIR3Z6RRUNN|          2|           50|         60|   Y|                N|0.8333333333333334|
|R2YO2RESXZEWOF|          4|           24|         25|   Y|                N|              0.96|
| R8GEE2EW3D6S9|          3|           49|         54|   Y|                N|0.9074074074074074|
|R33W9V1MX6WKPS|          4|           47|         53|   Y|                N|0.8867924528301887|
|R24E3W2ZRIFG8C|          5|           91|         93|   Y|                N| 0.978494623655914|
|R2OR3KIFFG4YWT|          4|           75|         81|   Y|                Y|0.9259259259259259|
|R1RLJAAFOA5MMS|          2|           25|         27|   Y|                N|0.9259259259259259|
| RVYHD7IZ7W99B|          1|  

In [40]:
# Step 4: Retrieves all the rows where the review was not part of the Vine program (unpaid)
non_paid_df = percent_votes_df.filter(col("vine") == "N").show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|0.6923076923076923|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|0.6153846153846154|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|              0.64|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|0.9344262295081968|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|               1.0|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|0.9072164948453608|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|0.6744186046511628|
|R39OOYN0DOLINK|          3|  

In [41]:
# Step 5: Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

from pyspark.sql.functions import col,when,count,lit
ratings_total_df = percent_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")).show()

+----+-------------+--------------------+------------------+
|vine|Total_Reviews|Total_5_Star_Reviews| %_5_Star_To_Total|
+----+-------------+--------------------+------------------+
|   Y|          248|                 102| 41.12903225806452|
|   N|        17513|                5153|29.423856563695537|
+----+-------------+--------------------+------------------+



In [43]:
ratings_total_df = vine_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("%_5_Star_To_Total")).show()

+----+-------------+--------------------+------------------+
|vine|Total_Reviews|Total_5_Star_Reviews| %_5_Star_To_Total|
+----+-------------+--------------------+------------------+
|   Y|        10415|                3381|32.462794047047524|
|   N|       331498|              150557|  45.4171669210674|
+----+-------------+--------------------+------------------+

